# Calculating molecular descriptors using RDKIT and Mordred

In [1]:
from rdkit.Chem import AllChem
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors

import pandas as pd

import numpy as np

from mordred import Calculator, descriptors

## 1. Generate canonical SMILES

In [2]:
def canonical_smiles(smiles):
    mols = [Chem.MolFromSmiles(smi) for smi in smiles]
    smiles = [Chem.MolToSmiles(mol) for mol in mols]
    return smiles

In [3]:
dataset = pd.read_excel('../data/bcl-xl_1.xlsx')

/Users/shark/opt/anaconda3/envs/ISDD/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [4]:
dataset2 = dataset.dropna(axis=1)

In [5]:
dataset2.head()

,PUBCHEM_SID,PUBCHEM_CID,PUBCHEM_ACTIVITY_OUTCOME,PUBCHEM_ACTIVITY_SCORE,Inhibition(10.9uM),SMILES,Norm_Act
0,57260153,2202,Active,8,30.78,C1C2=C(C(=CC=C2)O)C(=O)C3=C1C=CC=C3O,0.385246
1,50085845,2215,Active,9,33.24,CN1CCC2=CC=CC3=C2C1CC4=C3C(=C(C=C4)O)O,0.389821
2,57265471,2259,Active,20,75.53,C1=CC(=C(C=C1C(=C2C=CC(=O)C(=C2)C(=O)O)C3=CC(=...,0.468462
3,855829,2794,Active,21,78.00,CC(C)N=C1C=C2C(=NC3=CC=CC=C3N2C4=CC=C(C=C4)Cl)...,0.473055
4,26657972,2866,Active,10,36.96,COC(=O)C1C(CCC2C1CC3C4=C(CCN3C2)C5=CC=CC=C5N4)O,0.396738


In [6]:
Canon_SMILES = canonical_smiles(dataset2.SMILES)

In [7]:
len(Canon_SMILES)
dataset2['SMILES'] = Canon_SMILES
dataset2

/var/folders/nn/1kbnq3455vv8xm9r3dx6ybx40000gn/T/ipykernel_31479/521908446.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset2['SMILES'] = Canon_SMILES


,PUBCHEM_SID,PUBCHEM_CID,PUBCHEM_ACTIVITY_OUTCOME,PUBCHEM_ACTIVITY_SCORE,Inhibition(10.9uM),SMILES,Norm_Act
0,57260153,2202,Active,8,30.78,O=C1c2c(O)cccc2Cc2cccc(O)c21,0.385246
1,50085845,2215,Active,9,33.24,CN1CCc2cccc3c2C1Cc1ccc(O)c(O)c1-3,0.389821
2,57265471,2259,Active,20,75.53,O=C(O)C1=CC(=C(c2ccc(O)c(C(=O)O)c2)c2ccc(O)c(C...,0.468462
3,855829,2794,Active,21,78.00,CC(C)N=c1cc2n(-c3ccc(Cl)cc3)c3ccccc3nc-2cc1Nc1...,0.473055
4,26657972,2866,Active,10,36.96,COC(=O)C1C(O)CCC2CN3CCc4c([nH]c5ccccc45)C3CC21,0.396738
...,...,...,...,...,...,...,...
995,49645508,5311,Inactive,0,2.67,O=C(CCCCCCC(=O)Nc1ccccc1)NO,0.332974
996,855781,5315,Inactive,0,3.17,O=C(O)CCC(=O)Nc1ccc(S(=O)(=O)Nc2nccs2)cc1,0.333904
997,855720,5319,Inactive,1,4.23,Nc1ccc(S(=O)(=O)NC(=O)c2ccccc2)cc1,0.335875
998,56422206,5320,Inactive,1,5.07,CC(=O)NS(=O)(=O)c1ccc(N)cc1,0.337437


In [8]:
duplicate_smiles = dataset2[dataset2['SMILES'].duplicated()]['SMILES'].values

In [9]:
len(duplicate_smiles)

0

In [10]:
dataset2[dataset2['SMILES'].isin(duplicate_smiles)].sort_values(by=['SMILES'])

,PUBCHEM_SID,PUBCHEM_CID,PUBCHEM_ACTIVITY_OUTCOME,PUBCHEM_ACTIVITY_SCORE,Inhibition(10.9uM),SMILES,Norm_Act


## 2. Drop duplicate values

In [11]:
dataset_new = dataset2.drop_duplicates(subset=['SMILES'])

In [12]:
dataset_new

,PUBCHEM_SID,PUBCHEM_CID,PUBCHEM_ACTIVITY_OUTCOME,PUBCHEM_ACTIVITY_SCORE,Inhibition(10.9uM),SMILES,Norm_Act
0,57260153,2202,Active,8,30.78,O=C1c2c(O)cccc2Cc2cccc(O)c21,0.385246
1,50085845,2215,Active,9,33.24,CN1CCc2cccc3c2C1Cc1ccc(O)c(O)c1-3,0.389821
2,57265471,2259,Active,20,75.53,O=C(O)C1=CC(=C(c2ccc(O)c(C(=O)O)c2)c2ccc(O)c(C...,0.468462
3,855829,2794,Active,21,78.00,CC(C)N=c1cc2n(-c3ccc(Cl)cc3)c3ccccc3nc-2cc1Nc1...,0.473055
4,26657972,2866,Active,10,36.96,COC(=O)C1C(O)CCC2CN3CCc4c([nH]c5ccccc45)C3CC21,0.396738
...,...,...,...,...,...,...,...
995,49645508,5311,Inactive,0,2.67,O=C(CCCCCCC(=O)Nc1ccccc1)NO,0.332974
996,855781,5315,Inactive,0,3.17,O=C(O)CCC(=O)Nc1ccc(S(=O)(=O)Nc2nccs2)cc1,0.333904
997,855720,5319,Inactive,1,4.23,Nc1ccc(S(=O)(=O)NC(=O)c2ccccc2)cc1,0.335875
998,56422206,5320,Inactive,1,5.07,CC(=O)NS(=O)(=O)c1ccc(N)cc1,0.337437


# Calculate descriptors using RDkit

## a. General molecular descriptors

In [13]:
def RDkit_descriptors(smiles):
    mols = [Chem.MolFromSmiles(i) for i in smiles]
    calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
    desc_names = calc.GetDescriptorNames()

    Mol_descriptors = []
    for mol in mols:
        mol = Chem.AddHs(mol)
        descriptors = calc.CalcDescriptors(mol)
        Mol_descriptors.append(descriptors)
    return Mol_descriptors, desc_names

Mol_descriptors, desc_names = RDkit_descriptors(dataset_new['SMILES'])

In [14]:
df_with_200_descriptors = pd.DataFrame(Mol_descriptors, columns=desc_names)
df_with_200_descriptors

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,13.037559,-2.777477,13.037559,0.676101,0.617649,226.231,216.151,226.062994,84,0,...,0,0,0,0,0,0,0,0,0,0
1,9.234808,-3.702504,9.234808,0.343793,0.721339,267.328,250.192,267.125929,102,0,...,0,0,0,0,0,0,0,0,0,0
2,12.698307,-1.843794,12.698307,1.118607,0.454171,422.345,408.233,422.063782,156,0,...,0,0,0,0,0,0,0,0,0,0
3,9.402049,-3.913789,9.402049,0.024131,0.274930,473.407,451.231,472.122152,164,0,...,0,0,0,0,0,0,0,0,0,0
4,13.845410,-5.057938,13.845410,0.234264,0.772957,354.450,328.242,354.194343,138,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,12.888013,-4.591666,12.888013,0.675930,0.383220,264.325,244.165,264.147393,104,0,...,0,0,0,0,0,0,0,0,0,0
996,12.882890,-5.245301,12.882890,0.285139,0.694550,355.397,342.293,355.029663,122,0,...,0,1,0,0,0,1,0,0,0,0
997,12.783922,-5.446006,12.783922,0.270375,0.828903,276.317,264.221,276.056863,98,0,...,0,1,0,0,0,0,0,0,0,0
998,12.201115,-5.335671,12.201115,0.260717,0.682817,214.246,204.166,214.041213,76,0,...,0,1,0,0,0,0,0,0,0,0


# Calculate descriptors using Mordred

In [15]:
def All_Mordred_descriptors(data):
    calc = Calculator(descriptors, ignore_3D=False)
    mols = [Chem.MolFromSmiles(smi) for smi in data]

    df = calc.pandas(mols)
    return df

In [16]:
mordred_descriptors = All_Mordred_descriptors(dataset_new['SMILES'])

  7%|▋         | 73/1000 [00:06<00:57, 16.16it/s]

/Users/shark/opt/anaconda3/envs/ISDD/lib/python3.10/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


100%|██████████| 1000/1000 [00:56<00:00, 17.70it/s]


In [17]:
mordred_descriptors.shape

(1000, 1826)

In [19]:
mordred_descriptors

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,13.520558,11.120921,0,0,21.807619,2.502393,5.004787,21.807619,1.282801,3.792682,...,10.043206,50.617718,226.062994,8.372703,450,31,94.0,115.0,5.527778,3.666667
1,16.268104,12.543031,0,1,26.925574,2.561599,5.123198,26.925574,1.346279,3.973787,...,10.338091,54.819955,267.125929,7.219620,680,40,116.0,145.0,6.0,4.222222
2,23.955071,19.821093,3,0,38.215107,2.468488,4.936976,38.215107,1.232745,4.351528,...,10.413523,67.009973,422.063782,9.379195,2484,54,162.0,192.0,12.694444,6.750000
3,26.357869,19.029926,0,0,43.326931,2.534137,5.068274,43.326931,1.312937,4.442183,...,10.540461,69.560629,472.122152,8.584039,3032,56,180.0,214.0,9.583333,7.083333
4,21.068022,15.865154,0,1,35.095195,2.56476,5.074063,35.095195,1.349815,4.232996,...,10.512737,76.259864,354.194343,6.811430,1517,50,150.0,187.0,7.222222,5.583333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,13.653808,11.360579,0,0,23.780548,2.194709,4.389417,23.780548,1.251608,3.806227,...,9.069813,50.382261,264.147393,6.773010,954,20,82.0,86.0,6.583333,4.583333
996,17.554960,14.610439,1,0,28.157189,2.382168,4.751126,28.157189,1.224226,4.039363,...,9.812413,69.887445,355.029663,9.861935,1405,29,114.0,127.0,8.618056,5.041667
997,14.576703,12.442615,0,0,23.599707,2.37641,4.752819,23.599707,1.24209,3.858347,...,9.767725,52.192165,276.056863,8.905060,736,27,96.0,109.0,7.006944,4.152778
998,10.483892,9.735797,0,0,15.899324,2.353178,4.706355,15.899324,1.135666,3.533503,...,9.418086,45.458479,214.041213,8.918384,307,18,68.0,75.0,6.645833,3.041667


In [25]:
concat = pd.concat([dataset_new, mordred_descriptors, df_with_200_descriptors], axis = 1)

In [26]:
concat

,PUBCHEM_SID,PUBCHEM_CID,PUBCHEM_ACTIVITY_OUTCOME,PUBCHEM_ACTIVITY_SCORE,Inhibition(10.9uM),SMILES,Norm_Act,ABC,ABCGG,nAcid,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,57260153,2202,Active,8,30.78,O=C1c2c(O)cccc2Cc2cccc(O)c21,0.385246,13.520558,11.120921,0,...,0,0,0,0,0,0,0,0,0,0
1,50085845,2215,Active,9,33.24,CN1CCc2cccc3c2C1Cc1ccc(O)c(O)c1-3,0.389821,16.268104,12.543031,0,...,0,0,0,0,0,0,0,0,0,0
2,57265471,2259,Active,20,75.53,O=C(O)C1=CC(=C(c2ccc(O)c(C(=O)O)c2)c2ccc(O)c(C...,0.468462,23.955071,19.821093,3,...,0,0,0,0,0,0,0,0,0,0
3,855829,2794,Active,21,78.00,CC(C)N=c1cc2n(-c3ccc(Cl)cc3)c3ccccc3nc-2cc1Nc1...,0.473055,26.357869,19.029926,0,...,0,0,0,0,0,0,0,0,0,0
4,26657972,2866,Active,10,36.96,COC(=O)C1C(O)CCC2CN3CCc4c([nH]c5ccccc45)C3CC21,0.396738,21.068022,15.865154,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,49645508,5311,Inactive,0,2.67,O=C(CCCCCCC(=O)Nc1ccccc1)NO,0.332974,13.653808,11.360579,0,...,0,0,0,0,0,0,0,0,0,0
996,855781,5315,Inactive,0,3.17,O=C(O)CCC(=O)Nc1ccc(S(=O)(=O)Nc2nccs2)cc1,0.333904,17.554960,14.610439,1,...,0,1,0,0,0,1,0,0,0,0
997,855720,5319,Inactive,1,4.23,Nc1ccc(S(=O)(=O)NC(=O)c2ccccc2)cc1,0.335875,14.576703,12.442615,0,...,0,1,0,0,0,0,0,0,0,0
998,56422206,5320,Inactive,1,5.07,CC(=O)NS(=O)(=O)c1ccc(N)cc1,0.337437,10.483892,9.735797,0,...,0,1,0,0,0,0,0,0,0,0


In [27]:
concat.to_csv("../data/table_1.csv", sep=";")